In [ ]:
# Day 6 - Warmup 1

import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
    'user_id': range(1, 21),
    'age': np.random.randint(18, 70, 20),
    'revenue': np.random.uniform(200, 2000, 20).round(2),
    'country': np.random.choice(['US', 'CA', 'UK', 'AU'], 20),
    'is_active': np.random.choice([True, False], 20)
})

In [ ]:
df['high_value'] = (df['revenue'] > 1000) & df['is_active'] 

df['target_segment'] = ((df['age'] < 30) | (df['age'] > 60)) & df['country'].isin(['US', 'CA'])

df.loc[df['high_value'] | (df['target_segment'] & (df['revenue'] > 500))]

# Rewrite the above using .query instead
filtered_df = df.query('high_value == True or (target_segment == True and revenue > 500)')
filtered_df

# Grade: A-

In [ ]:
# Warmup 2 datetime index manipulation 

import pandas as pd
import numpy as np

np.random.seed(42)

dates = pd.date_range('2025-01-01', periods=30, freq='D')
df = pd.DataFrame({
    'date': dates,
    'sales': np.random.randint(100, 500, 30),
    'visits': np.random.randint(50, 200, 30)
})

In [ ]:
df = df.set_index('date')

df['day_of_week'] = df.index.dayofweek

df['week_of_year'] = df.index.isocalendar().week

df['is_weekend'] = df['day_of_week'] >= 5



# I've never tried to do a centered window before so this is just a regular one

df['sales_7d_avg'] = df.rolling('7d', min_periods= 7, center=True)['sales'].mean()

# Grade: A-

In [1]:
# Warmup 3 - Broadcasting and Normalisation 
import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
    'user_id': ['U1', 'U2', 'U3', 'U4', 'U5'],
    'math_score': [85, 92, 78, 88, 95],
    'reading_score': [80, 85, 90, 82, 88],
    'science_score': [88, 91, 85, 90, 92]
})

In [3]:
df['total_score'] = df[['math_score', 'reading_score', 'science_score']].sum(axis = 1)



df['math_pct'] = df['math_score']/df['total_score']

df['reading_pct'] = df['reading_score']/df['total_score']

df['science_pct'] = df['science_score']/df['total_score']



df['math_zscore'] = (df['math_score'] - df['math_score'].mean())/df['math_score'].std()

df['reading_zscore'] = (df['reading_score'] - df['reading_score'].mean())/df['reading_score'].std()

df['science_zscore'] = (df['science_score'] - df['science_score'].mean())/df['science_score'].std()

df.head()

,user_id,math_score,reading_score,science_score,total_score,math_pct,reading_pct,science_pct,math_zscore,reading_zscore,science_zscore
0,U1,85,80,88,253,0.335968,0.316206,0.347826,-0.395120,-1.212678,-0.432450
1,U2,92,85,91,268,0.343284,0.317164,0.339552,0.668665,0.000000,0.648675
2,U3,78,90,85,253,0.308300,0.355731,0.335968,-1.458906,1.212678,-1.513575
3,U4,88,82,90,260,0.338462,0.315385,0.346154,0.060788,-0.727607,0.288300
4,U5,95,88,92,275,0.345455,0.320000,0.334545,1.124573,0.727607,1.009050


# Grade: A

In [17]:
# Potential optimisations
score_cols = ['math_score', 'reading_score', 'science_score']
pct_cols = ['math_pct', 'reading_pct', 'science_pct']

df[pct_cols] = df[score_cols].div(df['total_score'], axis=0)

KeyError: "None of [Index(['math_score', 'reading_score', 'science_score'], dtype='object')] are in the [columns]"

In [ ]:
# Potential optimisations
from scipy.stats import zscore

df[['math_zscore', 'reading_zscore', 'science_zscore']] = df[score_cols].apply(zscore)

In [ ]:
# Warmup 4 - Strings and Regex 

import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
    'user_id': ['U001', 'U002', 'U003', 'U004', 'U005'],
    'email': [
        'john.doe@company.com',
        'jane_smith@gmail.com',
        'invalid-email',
        'bob.jones@company.co.uk',
        'alice@subdomain.example.org'
    ],
    'phone': [
        '555-1234',
        '555.5678',
        '5559012',
        '555-3456',
        '(555) 7890'
    ]
})

Create a column email_domain that extracts the domain from the email (everything after the @). If there's no @, set it to None.
Create a column is_valid_email that is True if the email contains:

- At least one character before @
- An @ symbol
- At least one . after the @

Otherwise False.

Create a column phone_clean that extracts only the digits from the phone column (remove all non-digit characters).
Create a column phone_formatted that formats phone_clean as XXX-XXXX (assumes all phones are 7 digits after cleaning). If phone_clean is not exactly 7 digits, set to None.

In [ ]:
df.head()

,user_id,email,phone
0,U001,john.doe@company.com,555-1234
1,U002,jane_smith@gmail.com,555.5678
2,U003,invalid-email,5559012
3,U004,bob.jones@company.co.uk,555-3456
4,U005,alice@subdomain.example.org,(555) 7890


In [ ]:
# First pass
df['email_domain'] = df["email"].str.extract(r".*\@(.+)")

df['is_valid_email'] = df['email'].str.contains(r".+\@.+")

df['phone_clean'] = df['phone'].str.replace(r'\D', '', regex=True)

df['phone_formatted'] = df['phone_clean'].str.replace(r'(^\d{4})(\d{3}$)', r'\1-\2', regex=True)

df.loc[~df['phone_formatted'].str.match(r'^\d{4}-\d{3}$'), 'phone_formatted'] = np.nan

In [ ]:
df.head()

,user_id,email,phone,email_domain,is_valid_email,phone_clean,phone_formatted
0,U001,john.doe@company.com,555-1234,company.com,True,5551234,5551-234
1,U002,jane_smith@gmail.com,555.5678,gmail.com,True,5555678,5555-678
2,U003,invalid-email,5559012,NaN,False,5559012,5559-012
3,U004,bob.jones@company.co.uk,555-3456,company.co.uk,True,5553456,5553-456
4,U005,alice@subdomain.example.org,(555) 7890,subdomain.example.org,True,5557890,5557-890


In [ ]:
df['email_domain'] = df['email'].str.extract(r'.*@(.+)')

# Fixed - search for an actual . after @, which was in the spec. 
df['is_valid_email'] = df['email'].str.contains(r'.+@.+\..+')

df['phone_clean'] = df['phone'].str.replace(r'\D', '', regex=True)

# Fixes - 3/4 not 4/3 formatting

# Option 1: Two-step (your approach, fixed)
df['phone_formatted'] = df['phone_clean'].str.replace(r'^(\d{3})(\d{4})$', r'\1-\2', regex=True)
df.loc[~df['phone_formatted'].str.match(r'^\d{3}-\d{4}$'), 'phone_formatted'] = None

# Option 2: One-step with .extract()
df['phone_formatted'] = df['phone_clean'].str.extract(r'^(\d{3})(\d{4})$').apply(
    lambda x: f'{x[0]}-{x[1]}' if x.notna().all() else None, axis=1
)

In [19]:
# Intermediate 1. Join + Aggregate

import pandas as pd
import numpy as np

np.random.seed(42)

# Users table
users = pd.DataFrame({
    'user_id': ['U1', 'U2', 'U3', 'U4', 'U5'],
    'username': ['alice', 'bob', 'charlie', 'diana', 'eve'],
    'signup_date': pd.to_datetime(['2024-01-15', '2024-02-20', '2024-01-10', '2024-03-05', '2024-02-28'])
})

# Purchases table
purchases = pd.DataFrame({
    'purchase_id': range(1, 16),
    'user_id': ['U1', 'U1', 'U2', 'U1', 'U3', 'U2', 'U4', 'U1', 'U3', 'U2', 'U5', 'U3', 'U4', 'U2', 'U1'],
    'category': ['electronics', 'books', 'electronics', 'clothing', 'books', 
                 'electronics', 'clothing', 'books', 'electronics', 'books',
                 'clothing', 'clothing', 'electronics', 'clothing', 'electronics'],
    'amount': [299.99, 19.99, 450.00, 89.99, 24.99, 199.99, 120.00, 15.99, 
               350.00, 29.99, 75.00, 110.00, 399.99, 95.00, 599.99]
})

1) For each user, calculate:

- total_revenue: sum of all their purchase amounts
- n_purchases: count of their purchases
- avg_purchase: mean purchase amount


2) Rank users by total_revenue (highest = rank 1). Add this as a column revenue_rank.
3) Merge this aggregated data back to the users table (left join, so all users appear even if they have no purchases).
4) Fill missing values:
- total_revenue, n_purchases, avg_purchase → 0
- revenue_rank → NaN (leave as-is for users with no purchases)



Final output should be the users table with these added columns:
- total_revenue
- n_purchases
- avg_purchase
- revenue_rank

In [22]:
users.head()
# purchases.head()

,user_id,username,signup_date
0,U1,alice,2024-01-15
1,U2,bob,2024-02-20
2,U3,charlie,2024-01-10
3,U4,diana,2024-03-05
4,U5,eve,2024-02-28


In [ ]:
combined = users.merge(purchases, on = 'user_id', how = 'left')



combined = combined.groupby('user_id').agg(
    total_revenue = ("amount", 'sum'), 
    n_purchases= ("purchase_id", 'count'),
    avg_purchase= ("amount", 'mean')
)

combined['revenue_rank'] = combined['total_revenue'].rank(method = 'dense', ascending = False)

final = users.merge(combined, on = 'user_id', how = 'left')

final['total_revenue'] = final['total_revenue'].fillna(0)

final['n_purchases'] = final['n_purchases'].fillna(0)

final['avg_purchase'] = final['avg_purchase'].fillna(0)


print(final)

  user_id username signup_date  total_revenue  n_purchases  avg_purchase  \
0      U1    alice  2024-01-15        1025.95            5    205.190000   
1      U2      bob  2024-02-20         774.98            4    193.745000   
2      U3  charlie  2024-01-10         484.99            3    161.663333   
3      U4    diana  2024-03-05         519.99            2    259.995000   
4      U5      eve  2024-02-28          75.00            1     75.000000   

   revenue_rank  
0           1.0  
1           2.0  
2           4.0  
3           3.0  
4           5.0  


# Grade: A-

In [28]:
# Key issue - joining then aggregating doesn't work with count(). For users with no purchases, they'll get a row with NaN, and count() will return 1 instead of 0. 
# A better strategy is to aggregate first, then join.

# Aggregate purchases by user
purchase_agg = purchases.groupby('user_id').agg(
    total_revenue=('amount', 'sum'),
    n_purchases=('purchase_id', 'count'),
    avg_purchase=('amount', 'mean')
).reset_index()

# Rank by revenue
purchase_agg['revenue_rank'] = purchase_agg['total_revenue'].rank(method='dense', ascending=False)

# Merge back to users
final = users.merge(purchase_agg, on='user_id', how='left')

# Fill NaNs
final['total_revenue'] = final['total_revenue'].fillna(0)
final['n_purchases'] = final['n_purchases'].fillna(0)
final['avg_purchase'] = final['avg_purchase'].fillna(0)

In [40]:
# Day 6: Intermediate 2 - MultiIndex Time Features

import pandas as pd
import numpy as np

np.random.seed(42)

# Daily sales data for multiple stores
df = pd.DataFrame({
    'date': pd.date_range('2025-01-01', periods=90, freq='D').tolist() * 3,
    'store_id': ['S1']*90 + ['S2']*90 + ['S3']*90,
    'daily_sales': np.random.randint(1000, 5000, 270)
})

df = df.sort_values(['store_id', 'date']).reset_index(drop=True)

### Tasks:
1.	For each store, calculate: 
- weekly_sales: sum of daily_sales for each week (group by store + week)
- monthly_sales: sum of daily_sales for each month (group by store + month)
2.	Merge these weekly and monthly totals back to the daily-level DataFrame, so each daily row shows: 
- The week's total sales for that store
- The month's total sales for that store
3.	Create a column daily_pct_of_week that shows what percentage of the week's total sales came from that specific day.
4.	Create a column daily_pct_of_month that shows what percentage of the month's total sales came from that specific day.
Final output: Daily-level DataFrame with added columns:
- weekly_sales
- monthly_sales
- daily_pct_of_week
- daily_pct_of_month


In [33]:
weekly_sales = df.groupby('store_id').resample('W', on = 'date')['daily_sales'].sum() 
monthly_sales = df.groupby('store_id').resample('M', on = 'date')['daily_sales'].sum()

df = df.merge(weekly_sales.to_frame(name='weekly_sales'), left_on=['store_id', 'date'], right_index=True, how = 'left') 
df = df.merge(monthly_sales.to_frame(name='monthly_sales') , left_on=['store_id', 'date'], right_index=True, how = 'left')

df['daily_pct_of_week'] = df['daily_sales']/df['weekly_sales'] * 100

df['daily_pct_of_month'] = df['daily_sales']/df['monthly_sales'] * 100



print(df)

          date store_id  daily_sales  weekly_sales  monthly_sales  \
0   2025-01-01       S1         4174           NaN            NaN   
1   2025-01-02       S1         4507           NaN            NaN   
2   2025-01-03       S1         1860           NaN            NaN   
3   2025-01-04       S1         2294           NaN            NaN   
4   2025-01-05       S1         2130       14965.0            NaN   
..         ...      ...          ...           ...            ...   
265 2025-03-27       S3         4445           NaN            NaN   
266 2025-03-28       S3         4743           NaN            NaN   
267 2025-03-29       S3         2631           NaN            NaN   
268 2025-03-30       S3         2495       23873.0            NaN   
269 2025-03-31       S3         4304           NaN        95890.0   

     daily_pct_of_week  daily_pct_of_month  
0                  NaN                 NaN  
1                  NaN                 NaN  
2                  NaN              

/tmp/ipykernel_70548/1237979520.py:2: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.groupby('store_id').resample('M', on = 'date')['daily_sales'].sum()


In [36]:
# Create period columns in your original df
df['week'] = df['date'].dt.to_period('W')
df['month'] = df['date'].dt.to_period('M')

# Resample and reset index to get period columns
weekly_sales = df.groupby('store_id').resample('W', on='date')['daily_sales'].sum().reset_index()
weekly_sales['week'] = weekly_sales['date'].dt.to_period('W')

monthly_sales = df.groupby('store_id').resample('M', on='date')['daily_sales'].sum().reset_index()
monthly_sales['month'] = monthly_sales['date'].dt.to_period('M')

# Merge on store + period
df = df.merge(weekly_sales[['store_id', 'week', 'daily_sales']], 
              on=['store_id', 'week'], 
              suffixes=('', '_weekly'))
df = df.merge(monthly_sales[['store_id', 'month', 'daily_sales']], 
              on=['store_id', 'month'], 
              suffixes=('', '_monthly'))

print(df)

          date store_id  daily_sales                   week    month  \
0   2025-01-01       S1         4174  2024-12-30/2025-01-05  2025-01   
1   2025-01-02       S1         4507  2024-12-30/2025-01-05  2025-01   
2   2025-01-03       S1         1860  2024-12-30/2025-01-05  2025-01   
3   2025-01-04       S1         2294  2024-12-30/2025-01-05  2025-01   
4   2025-01-05       S1         2130  2024-12-30/2025-01-05  2025-01   
..         ...      ...          ...                    ...      ...   
265 2025-03-27       S3         4445  2025-03-24/2025-03-30  2025-03   
266 2025-03-28       S3         4743  2025-03-24/2025-03-30  2025-03   
267 2025-03-29       S3         2631  2025-03-24/2025-03-30  2025-03   
268 2025-03-30       S3         2495  2025-03-24/2025-03-30  2025-03   
269 2025-03-31       S3         4304  2025-03-31/2025-04-06  2025-03   

     daily_sales_weekly  daily_sales_monthly  daily_sales_weekly  \
0                 14965                94786               14965   

/tmp/ipykernel_70548/1189760977.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.groupby('store_id').resample('M', on='date')['daily_sales'].sum().reset_index()


# Grade: B

- Didn't need resample after groupby
- Didn't rename columns to the spec provided 
- Didn't use transform which is cleaner here. 

In [41]:
# Cleaner pattern: 

df['week'] = df['date'].dt.to_period('W')
df['month'] = df['date'].dt.to_period('M')

df['weekly_sales'] = df.groupby(['store_id', 'week'])['daily_sales'].transform('sum')
df['monthly_sales'] = df.groupby(['store_id', 'month'])['daily_sales'].transform('sum')

df['daily_pct_of_week'] = df['daily_sales'] / df['weekly_sales']
df['daily_pct_of_month'] = df['daily_sales'] / df['monthly_sales']
print(df)

          date store_id  daily_sales                   week    month  \
0   2025-01-01       S1         4174  2024-12-30/2025-01-05  2025-01   
1   2025-01-02       S1         4507  2024-12-30/2025-01-05  2025-01   
2   2025-01-03       S1         1860  2024-12-30/2025-01-05  2025-01   
3   2025-01-04       S1         2294  2024-12-30/2025-01-05  2025-01   
4   2025-01-05       S1         2130  2024-12-30/2025-01-05  2025-01   
..         ...      ...          ...                    ...      ...   
265 2025-03-27       S3         4445  2025-03-24/2025-03-30  2025-03   
266 2025-03-28       S3         4743  2025-03-24/2025-03-30  2025-03   
267 2025-03-29       S3         2631  2025-03-24/2025-03-30  2025-03   
268 2025-03-30       S3         2495  2025-03-24/2025-03-30  2025-03   
269 2025-03-31       S3         4304  2025-03-31/2025-04-06  2025-03   

     weekly_sales  monthly_sales  daily_pct_of_week  daily_pct_of_month  
0           14965          94786           0.278917          

In [42]:
# Day 6 — Intermediate 3: Binning & Categorization
import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
    'student_id': range(1, 21),
    'exam_score': [45, 67, 89, 72, 91, 58, 77, 83, 65, 94, 
                   50, 88, 76, 62, 85, 70, 93, 55, 80, 68],
    'age': [18, 19, 20, 18, 21, 19, 20, 22, 18, 21,
            19, 20, 18, 19, 21, 20, 22, 19, 20, 18]
})

In [ ]:
# Default is right = True, which means 'ignore the lowest number and inclusive of the highest number in each bin'. 
# Setting right = False means 'include the lowest number and ignore the highest number in each bin'. So for example, with right = False, 
# a score of 60 would be included in the 'D' bin (60-70) rather than the 'F' bin (0-60).

df['letter_grade'] = pd.cut(df['exam_score'], bins = [0, 60, 70, 80, 90, 101], labels = ['F', 'D', 'C', 'B', 'A'], include_lowest=True, right = False)

df['score_tier'] = pd.cut(df['exam_score'], bins = 3, labels = ['low', 'medium', 'high'], include_lowest=True)
df['score_quartile'] = pd.qcut(df['exam_score'], q = 4, labels = ['Q1', 'Q2', 'Q3', 'Q4'])
df['age_group'] = pd.cut(df['age'], bins = [18, 20, 22, 100], labels = ['18-19', '20-21', '22+'], right = False)



print(df)

    student_id  exam_score  age letter_grade score_tier score_quartile  \
0            1          45   18            F        low             Q1   
1            2          67   19            D     medium             Q2   
2            3          89   20            B       high             Q4   
3            4          72   18            C     medium             Q2   
4            5          91   21            A       high             Q4   
5            6          58   19            F        low             Q1   
6            7          77   20            C     medium             Q3   
7            8          83   22            B       high             Q3   
8            9          65   18            D     medium             Q2   
9           10          94   21            A       high             Q4   
10          11          50   19            F        low             Q1   
11          12          88   20            B       high             Q4   
12          13          76   18       

# Grade: A

In [47]:
# Day 6 — Intermediate 4: Lagged Panel Features

import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
    'user_id': ['U1', 'U1', 'U1', 'U1', 'U1',
                'U2', 'U2', 'U2', 'U2', 'U2',
                'U3', 'U3', 'U3', 'U3', 'U3'],
    'date': pd.to_datetime([
        '2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04', '2025-01-05',
        '2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04', '2025-01-05',
        '2025-01-01', '2025-01-02', '2025-01-03', '2025-01-04', '2025-01-05'
    ]),
    'revenue': [100, 150, 120, 180, 200,
                80, 90, 110, 95, 105,
                200, 210, 190, 220, 230]
})

In [52]:
df.sort_values(['user_id', 'date']).reset_index(drop=True)['date']

0    2025-01-01
1    2025-01-02
2    2025-01-03
3    2025-01-04
4    2025-01-05
5    2025-01-01
6    2025-01-02
7    2025-01-03
8    2025-01-04
9    2025-01-05
10   2025-01-01
11   2025-01-02
12   2025-01-03
13   2025-01-04
14   2025-01-05
Name: date, dtype: datetime64[ns]

In [55]:
df = df.sort_values(['user_id', 'date']).reset_index(drop=True)

df['revenue_lag1'] = df.groupby('user_id')['revenue'].shift(1)

df['revenue_lag2'] = df.groupby('user_id')['revenue'].shift(2)

df['revenue_rolling_3d'] = df.groupby('user_id').rolling('3D', on = 'date')['revenue'].mean().reset_index(drop=True)

df['revenue_change'] = df['revenue'] - df['revenue_lag1']



print(df)

   user_id       date  revenue  revenue_lag1  revenue_lag2  \
0       U1 2025-01-01      100           NaN           NaN   
1       U1 2025-01-02      150         100.0           NaN   
2       U1 2025-01-03      120         150.0         100.0   
3       U1 2025-01-04      180         120.0         150.0   
4       U1 2025-01-05      200         180.0         120.0   
5       U2 2025-01-01       80           NaN           NaN   
6       U2 2025-01-02       90          80.0           NaN   
7       U2 2025-01-03      110          90.0          80.0   
8       U2 2025-01-04       95         110.0          90.0   
9       U2 2025-01-05      105          95.0         110.0   
10      U3 2025-01-01      200           NaN           NaN   
11      U3 2025-01-02      210         200.0           NaN   
12      U3 2025-01-03      190         210.0         200.0   
13      U3 2025-01-04      220         190.0         210.0   
14      U3 2025-01-05      230         220.0         190.0   

    rev

# Grade: A

In [56]:
# Day 6 — Intermediate 5: Resample vs Groupby

import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
    'date': pd.date_range('2025-01-01', periods=30, freq='D'),
    'category': ['A', 'B'] * 15,
    'sales': np.random.randint(100, 500, 30)
})

In [62]:
df = df.sort_values(['date', 'category'])

resample_weekly = df.resample('W', on = 'date')['sales'].sum()

resample_groupby = df.groupby(df['date'].dt.to_period('W'))['sales'].sum()

df['week'] = df['date'].dt.to_period('W')

weekly_categories = df.groupby(['week', 'category'])['sales'].sum().reset_index()

weekly_categories['weekly_pct'] = weekly_categories.groupby('week')['sales'].transform(lambda x: x/x.sum())


print(weekly_categories)

                    week category  sales  weekly_pct
0  2024-12-30/2025-01-05        A    743    0.531854
1  2024-12-30/2025-01-05        B    654    0.468146
2  2025-01-06/2025-01-12        A    771    0.437571
3  2025-01-06/2025-01-12        B    991    0.562429
4  2025-01-13/2025-01-19        A   1569    0.691799
5  2025-01-13/2025-01-19        B    699    0.308201
6  2025-01-20/2025-01-26        A   1304    0.500576
7  2025-01-20/2025-01-26        B   1301    0.499424
8  2025-01-27/2025-02-02        A    765    0.626536
9  2025-01-27/2025-02-02        B    456    0.373464


# Grade: A

In [78]:
# Day 6 — Hard 1: Funnel Analysis

import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
    'user_id': ['U1', 'U1', 'U1', 'U1',
                'U2', 'U2', 'U2',
                'U3', 'U3',
                'U4', 'U4', 'U4', 'U4',
                'U5', 'U5', 'U5'],
    'event_date': pd.to_datetime([
        '2025-01-01', '2025-01-01', '2025-01-02', '2025-01-03',
        '2025-01-01', '2025-01-02', '2025-01-03',
        '2025-01-01', '2025-01-02',
        '2025-01-01', '2025-01-01', '2025-01-02', '2025-01-03',
        '2025-01-01', '2025-01-02', '2025-01-02'
    ]),
    'event_type': [
        'page_view', 'add_to_cart', 'add_to_cart', 'purchase',
        'page_view', 'add_to_cart', 'purchase',
        'page_view', 'page_view',
        'page_view', 'add_to_cart', 'add_to_cart', 'purchase',
        'page_view', 'add_to_cart', 'purchase'
    ]
})

In [75]:
df = df.sort_values(['user_id', 'event_date'])



# Note - drop_duplicates is not a groupby operation as initially though. Using subset argument gives the expected groupby functionality. 



event_counts = df.drop_duplicates(subset=['user_id', 'event_type'], keep='first')['event_type'].value_counts()
event_counts = event_counts.reset_index(name = 'count').rename(columns = {'index': 'event_type'})

page_to_cart = event_counts.loc[event_counts['event_type'] == 'add_to_cart', 'count'].iloc[0] / event_counts.loc[event_counts['event_type'] == 'page_view', 'count'].iloc[0]
cart_to_purchase = event_counts.loc[event_counts['event_type'] == 'purchase', 'count'].iloc[0] / event_counts.loc[event_counts['event_type'] == 'add_to_cart', 'count'].iloc[0]
page_to_purchase = event_counts.loc[event_counts['event_type'] == 'purchase', 'count'].iloc[0] / event_counts.loc[event_counts['event_type'] == 'page_view', 'count'].iloc[0]

In [88]:
df =  df.drop_duplicates(subset=['user_id', 'event_type'], keep='first')

df_wide = df.pivot_table(index = 'user_id', columns = 'event_type', values='event_date').reset_index()
df_wide['valid_funnel'] = (df_wide['purchase'] > df_wide['add_to_cart']) & (df_wide['add_to_cart'] > df_wide['page_view'])

df_wide

event_type,user_id,add_to_cart,page_view,purchase,valid_funnel
0,U1,2025-01-01,2025-01-01,2025-01-03,False
1,U2,2025-01-02,2025-01-01,2025-01-03,True
2,U3,NaT,2025-01-01,NaT,False
3,U4,2025-01-01,2025-01-01,2025-01-03,False
4,U5,2025-01-02,2025-01-01,2025-01-02,False


In [91]:
df_summary = pd.DataFrame({
    'funnel_stage': ['page_view', 'add_to_cart', 'purchase'],
    'count': [df_wide['page_view'].count(), df_wide['add_to_cart'].count(), df_wide['purchase'].count()],
    'conversion_rate': [np.nan, page_to_cart, cart_to_purchase]
})

df_summary

,funnel_stage,count,conversion_rate
0,page_view,5,NaN
1,add_to_cart,4,0.8
2,purchase,4,1.0


# Grade: B+

In [ ]:
# Coach corrected verson. Main difference is using a cleaner way to get counts rather than value_counts and then havnig to use iloc repeatedly. 

# Get first occurrence of each event per user
df_first = df.sort_values(['user_id', 'event_date']).drop_duplicates(subset=['user_id', 'event_type'], keep='first')

# Pivot to wide
df_wide = df_first.pivot_table(index='user_id', columns='event_type', values='event_date').reset_index()

# Valid funnel (all steps present AND in order)
df_wide['valid_funnel'] = (
    df_wide['page_view'].notna() &
    df_wide['add_to_cart'].notna() &
    df_wide['purchase'].notna() &
    (df_wide['page_view'] < df_wide['add_to_cart']) &
    (df_wide['add_to_cart'] < df_wide['purchase'])
)

# Count users at each step
counts = df_wide[['page_view', 'add_to_cart', 'purchase']].notna().sum()

# Summary DataFrame
df_summary = pd.DataFrame({
    'funnel_step': ['page_view', 'add_to_cart', 'purchase'],
    'n_users': [counts['page_view'], counts['add_to_cart'], counts['purchase']]
})

df_summary['conversion_rate'] = df_summary['n_users'] / df_summary['n_users'].shift(1)

In [92]:
# Day 6 — Hard 2: Cohort + Rolling Retention Hybrid

import pandas as pd
import numpy as np

np.random.seed(42)

df = pd.DataFrame({
    "user": [
        "U1","U1","U1","U1",
        "U2","U2","U2",
        "U3","U3",
        "U4","U4","U4"
    ],
    "event_date": pd.to_datetime([
        "2025-01-01","2025-01-03","2025-01-10","2025-01-20",
        "2025-01-02","2025-01-08","2025-01-18",
        "2025-01-05","2025-01-25",
        "2025-01-07","2025-01-09","2025-01-15"
    ]),
    "event_type": [
        "visit","visit","purchase","visit",
        "visit","visit","purchase",
        "visit","visit",
        "visit","purchase","visit"
    ]
})

In [99]:
df = df.sort_values(['user', 'event_date'])

df['first_date'] = df.groupby('user')['event_date'].transform('min')
df['cohort_week'] = df['first_date'].dt.to_period('W')

purchases = df.loc[df['event_type'] == 'purchase', ['user', 'event_date']].rename(columns={'event_date': 'purchase_date'})

df_combine = df.merge(purchases, on = 'user')

df_combine['purchase_next_14d'] = (df_combine['purchase_date'] < df_combine['event_date'] + pd.Timedelta('14D')) & (df_combine['purchase_date'] > df_combine['event_date'])



df_combine['event_week'] = df_combine['event_date'].dt.to_period('W')



df_combine.groupby(['cohort_week', 'event_week']).agg(n_users = ('user', 'nunique'), retention_rate = ('purchase_next_14d', 'mean')).reset_index()

,cohort_week,event_week,n_users,retention_rate
0,2024-12-30/2025-01-05,2024-12-30/2025-01-05,2,0.666667
1,2024-12-30/2025-01-05,2025-01-06/2025-01-12,2,0.500000
2,2024-12-30/2025-01-05,2025-01-13/2025-01-19,1,0.000000
3,2024-12-30/2025-01-05,2025-01-20/2025-01-26,1,0.000000
4,2025-01-06/2025-01-12,2025-01-06/2025-01-12,1,0.500000
5,2025-01-06/2025-01-12,2025-01-13/2025-01-19,1,0.000000
